## Imports

In [1]:
import pandas as pd
import numpy as np

!pip3 install surprise
from surprise import Reader
from surprise import Dataset

from surprise import SVD, KNNBasic, NMF, SlopeOne, CoClustering #other knn, randoms, not svdpp because were not doing implicit               # importer ici les algo qu'on testera
from surprise import model_selection
from surprise import dump

from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeCV

In [2]:
def standardize(x):
    """Standardize the original data set."""
    mean_x = np.mean(x)
    x = x - mean_x
    std_x = np.std(x)
    x = x / std_x
    return x, mean_x, std_x

def expansion(x,degree=1):
    #polynomial expansion
    X_ex=x
    for deg in range(2, degree):
        X_ex = np.c_[X_ex, np.power(x, deg)]
    #cross term
    for i in range(0, x.shape[1]):
        for j in range(i+1, x.shape[1]):
            X_ex = np.c_[X_ex, np.multiply(x[:,i],x[:,j])]
    return X_ex

In [3]:
df = pd.read_csv("Datasets/data_train.csv")

df[["user", "item"]] = df.Id.str.split("_", expand=True)

df.user = df.user.str.replace("r", "")
df.item = df.item.str.replace("c", "")

#########
df2 = pd.read_csv("Datasets/sample_submission.csv")

df2[["user", "item"]] = df2.Id.str.split("_", expand=True)

df2.user = df2.user.str.replace("r", "")
df2.item = df2.item.str.replace("c", "")
prev_size=df.shape[0]
df

,Id,Prediction,user,item
0,r44_c1,4,44,1
1,r61_c1,3,61,1
2,r67_c1,4,67,1
3,r72_c1,3,72,1
4,r86_c1,5,86,1
...,...,...,...,...
1176947,r9990_c1000,4,9990,1000
1176948,r9992_c1000,5,9992,1000
1176949,r9994_c1000,3,9994,1000
1176950,r9997_c1000,4,9997,1000


## Loading des pickles

In [ ]:
_, algo_svd = dump.load("dump/dump_SVD")
_, algo_knn = dump.load("dump/dump_KNN_basic")
_, algo_nmf = dump.load("dump/dump_NMF")
_, algo_SlopeOne = dump.load("dump/dump_SlopeOne")
_, algo_coclustering = dump.load("dump/dump_CoClustering")


In [ ]:
def get_users(line):
    '''Get the userID'''
    row, col = line.split("_")
    row = row.replace("r", "")
    return str(row)

def get_items(line):
    '''get the movieID'''
    row, col = line.split("_")
    col = col.replace("c", "")
    return str(col)

def to_surprise(data):
    '''Move the dataframe from the Id/rating format to the userID/itemID/rating format'''
    data['userID'] = data['Id'].apply(get_users)
    data['itemID'] = data['Id'].apply(get_items)
    data = data.drop('Id', axis=1)
    data = data.rename(columns={'Prediction':'rating'})[['userID','itemID','rating']]
    return data


In [ ]:
test_original = pd.read_csv('Datasets/sample_submission.csv')
test = test_original.copy()
test = to_surprise(test)
test = Dataset.load_from_df(test, reader)
test = test.build_full_trainset()
test = test.build_testset()

In [ ]:
algo_svd.predict("37","1")

In [ ]:
array_SVD=algo_svd.test(test,verbose=False)
np.save("pred_svd_for_submission",array_SVD)
del array_SVD

In [ ]:
array_KNN=algo_knn.test(test)
np.save("pred_knn_for_submission",array_KNN)
del array_KNN

In [ ]:
array_NMF=algo_nmf.test(test)
np.save("pred_nmf_for_submission",array_NMF)
del array_NMF

In [ ]:
array_SlopeOne=algo_SlopeOne.test(test)
np.save("pred_slopeone_for_submission",array_SlopeOne)
del array_SlopeOne

In [ ]:
array_CoClustering=algo_coclustering.test(test)
np.save("pred_coclustering_for_submission",array_CoClustering)
del array_CoClustering

In [4]:
array_SVD=np.load("pred_svd_for_submission.npy")
array_KNN=np.load("pred_knn_for_submission.npy")
array_NMF=np.load("pred_nmf_for_submission.npy")
array_CoClustering=np.load("pred_coclustering_for_submission.npy")
array_SlopeOne=np.load("pred_slopeone_for_submission.npy")

In [5]:
tmp=np.vstack([array_SVD[:,3], array_KNN[:,3], array_NMF[:,3], array_SlopeOne[:,3],array_CoClustering[:,3]])

In [6]:
tmp=np.transpose(tmp)

In [7]:
del array_CoClustering
del array_KNN
del array_SlopeOne
del array_SVD
del array_NMF

# Contruction of matrix for ridge

In [ ]:
#Alix 08/12 replace this shit by the .test() method as above but on the trainset 
valid_items=df.item.unique()
valid_users=df.user.unique()
global_mean=df["Prediction"].mean()
count_bad_users=0
count_bad_items=0
count_bad_both=0
global_mean=df["Prediction"].mean()

for i in df.iterrows():
    if i[0]%100000==0:
        print(i[0])
    if (i[1]["user"] in valid_users and i[1]["item"] in valid_items):
        array_SVD[i[0]] = algo_svd.estimate(int(i[1][2])-1, int(i[1][3])-1)
        array_KNN[i[0]]= algo_knn.estimate(int(i[1][2])-1, int(i[1][3])-1)[0]
        array_NMF[i[0]] = algo_nmf.estimate(int(i[1][2])-1, int(i[1][3])-1)
        array_SlopeOne[i[0]]=algo_slopeone.estimate(int(i[1][2])-1, int(i[1][3])-1)
        array_CoClustering[i[0]] = algo_coclustering.estimate(int(i[1][2])-1, int(i[1][3])-1)
    elif (i[1]["user"] in valid_users):
        user_mean=df[df["user"]==i[1]["user"]].mean()
        array_SVD[i[0]] = user_mean
        array_KNN[i[0]]= user_mean
        array_NMF[i[0]] = user_mean
        array_SlopeOne[i[0]]=user_mean
        array_CoClustering[i[0]] = user_mean
        count_bad_users+=1
    elif (i[1]["item"] in valid_items):
        item_mean=df[df["item"]==i[1]["item"]].mean()
        array_SVD[i[0]] = item_mean
        array_KNN[i[0]]= item_mean
        array_NMF[i[0]] = item_mean
        array_SlopeOne[i[0]]= item_mean
        array_CoClustering[i[0]] = item_mean
        count_bad_items+=1
    else:
        array_SVD[i[0]] = global_mean
        array_KNN[i[0]]= global_mean
        array_NMF[i[0]] = global_mean
        array_SlopeOne[i[0]]= global_mean
        array_CoClustering[i[0]] = global_mean
        count_bad_both+=1
print(f"Done with {count_bad_users} bad users, {count_bad_items} bad items, {count_bad_both} bad both.")



In [ ]:
X = np.concatenate((array_SVD, array_KNN, array_NMF, array_SlopeOne, array_CoClustering), axis=1)
X[np.where(X>5)]=5
X[np.where(X<1)]=1
np.save("full_pred_array_in_range_1_5",X)

In [8]:
"""X_untouched = np.load("full_pred_array.npy")
print(np.where(X_untouched>5))
print(np.where(X_untouched<1))
"""
X=np.load("full_pred_array_in_range_1_5.npy")


## Data expansion

In [9]:
X_expand = expansion(X,4)
del X

In [10]:
y=np.array(df["Prediction"])
print(y)
clf=RidgeCV(alphas=np.linspace(10**-5,1,10),cv=10)

[4 3 4 ... 3 4 3]


In [11]:
X_expand=standardize(X_expand)[0]
clf=clf.fit(X_expand,y)

In [18]:
clf.alpha_
clf.coef_

array([ 3.56953879, -1.45354922,  3.72157772,  5.39914987,  2.1460635 ,
       -1.76588936,  6.90499038,  1.99524274,  3.15559124, -1.31329082,
        0.16726713, -0.61157989, -0.11697398,  0.19776078,  0.07545713,
        0.59147389,  0.76999595,  0.04250572, -0.50183058, -0.10444069,
       -0.70131696, -0.28465985, -8.25135512,  2.56837642, -0.54938787])

In [ ]:
#print(clf.cv_values_)
print(clf.coef_)
pred=np.rint(clf.predict(X_expand))
print(np.unique(pred,return_counts=True))
print(pred)
print(y)
len(np.where(y-pred==0.0)[0])

In [ ]:
"""import xgboost as xgb
from sklearn.model_selection import train_test_split

x_train, x_test= train_test_split(X_expand, test_size=0.2)



model = xgb.XGBClassifier()
model.fit(x_train, y_train)

print(model.feature_importances_)"""

# Estimations

In [ ]:
"""valid_items=df.item.unique()
valid_users=df.user.unique()
global_mean=df["Prediction"].mean()
count_bad_users=0
count_bad_items=0
count_bad_both=0
global_mean=df["Prediction"].mean()

for i in df2.iterrows():
    if i[0]%100000==0:
        print(i[0])
    if (i[1]["user"] in valid_users and i[1]["item"] in valid_items):
        array_SVD[i[0]] = algo_svd.estimate(int(i[1][2])-1, int(i[1][3])-1)
        array_KNN[i[0]]= algo_knn.estimate(int(i[1][2])-1, int(i[1][3])-1)[0]
        array_NMF[i[0]] = algo_nmf.estimate(int(i[1][2])-1, int(i[1][3])-1)
        array_SlopeOne[i[0]]=algo_SlopeOne.estimate(int(i[1][2])-1, int(i[1][3])-1)
        array_CoClustering[i[0]] = algo_coclustering.estimate(int(i[1][2])-1, int(i[1][3])-1)
    elif (i[1]["user"] in valid_users):
        user_mean=df[df["user"]==i[1]["user"]].mean()
        array_SVD[i[0]] = user_mean
        array_KNN[i[0]]= user_mean
        array_NMF[i[0]] = user_mean
        array_SlopeOne[i[0]]=user_mean
        array_CoClustering[i[0]] = user_mean
        count_bad_users+=1
    elif (i[1]["item"] in valid_items):
        item_mean=df[df["item"]==i[1]["item"]].mean()
        array_SVD[i[0]] = item_mean
        array_KNN[i[0]]= item_mean
        array_NMF[i[0]] = item_mean
        array_SlopeOne[i[0]]= item_mean
        array_CoClustering[i[0]] = item_mean
        count_bad_items+=1
    else:
        array_SVD[i[0]] = global_mean
        array_KNN[i[0]]= global_mean
        array_NMF[i[0]] = global_mean
        array_SlopeOne[i[0]]= global_mean
        array_CoClustering[i[0]] = global_mean
        count_bad_both+=1
print(f"Done with {count_bad_users} bad users, {count_bad_items} bad items, {count_bad_both} bad both.")

"""

### Blending 

In [13]:
#tmp=np.load("full_pred_array_in_range_1_5_for_submission.npy")
tmp=standardize(expansion(tmp,4))[0]
pred=clf.predict(tmp)
print(pred)

[3.28073502 3.34849562 3.44458121 ... 3.92069315 3.27649844 2.75553939]


In [14]:
final_array=np.rint(pred)
final_array[np.where(final_array>5)]=5
final_array[np.where(final_array<1)]=1
print(final_array)
print(np.unique(final_array, return_counts=True))

[3. 3. 3. ... 4. 3. 3.]
(array([2., 3., 4., 5.]), array([  7129, 308056, 858446,   3321]))


## Transformation en submission

In [15]:
df2.Prediction = final_array

In [16]:
df2 = df2.drop(columns=["user", "item"])

In [17]:
df2.to_csv("Datasets/submission_pipeline.csv", index=False)